In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

In [ ]:
# import getpass
# import os
# import time
from google.colab import userdata
# userdata.get('PINECONE_API_KEY')

from pinecone import Pinecone, ServerlessSpec

# if not os.getenv("PINECONE_API_KEY"):
#     os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
# import time

# index_name = "online-rag-project"  # change if desired

# pc.create_index(
#     name=index_name,
#     dimension=768,#model of google 768
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )
# index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd89611eef867ca78eba8266a82c9a543', 'Date': 'Thu, 02 Jan 2025 18:05:11 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec

# index_name = "langchain-test-index"  # change if desired
index_name = "online-rag-project"  # change if desired
existing_indexes = [index_info["name"] for index_info in pc.list_indexes().indexes] # corrected line

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        # dimension=3072,
        dimension=768,#model of google 768
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
vector = embeddings.embed_query("we are building a RAG Text")


In [ ]:
vector[:5]

[0.04060795158147812,
 -0.04385491460561752,
 -0.05683692544698715,
 -0.021194666624069214,
 0.010449625551700592]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [ ]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [ ]:
# data save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store.add_documents(documents=documents, ids=uuids)
# # data reterive

In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4
uuid4()

UUID('ba09f1a7-4986-42db-ac50-1543b5c51e73')

In [ ]:

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)
# data reterive

['5600ecf0-13e2-4754-bb70-ab4da83532fe',
 'e056ee47-538a-48d7-8f51-52c227207bb1',
 '7c8e175c-d87b-4bb6-8a40-0fb47896d473',
 'cd32d4b4-6fb2-48b3-8196-6e35bceb9b35',
 'd266bb57-84af-4398-9a58-7e42d3b5c6d8',
 'e8e3485b-dcae-4484-bb86-3d3741d8c94c',
 '766dff42-53bb-4d7c-b601-6d60b9b6ec57',
 'b10bfe68-a8eb-4d36-afad-e09ceabf4828',
 'e83b862e-4ae5-4f25-b397-9726413805de',
 '7a607f46-9c55-4d90-a353-5f74df1d30e3']

In [ ]:
# data reterive
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?"#, k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577411] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.537620] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.534239] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [ ]:
def answer_to_user(query: str) -> str:
  #vector search
   results = vector_store.similarity_search(query, k=2)
    # results = vector_store.similarity_search_with_score(query, k=1)
    #pass to model vector results + user query
   final_answer = model(result, query)
   return final_answer